In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset



In [5]:

class DiabetesDataset(Dataset):
    def __init__(self, csv_file):
        # Load CSV data
        self.df = pd.read_csv(csv_file)
        
        # Extract state features: 8 dimensions
        # [glu, glu_d, glu_t, hr, hr_d, hr_t, iob, hour_norm]
        self.states = self.df[[
            "glu", "glu_d", "glu_t",
            "hr", "hr_d", "hr_t",
            "iob", "hour"
        ]].values.astype(np.float32)
        
        # Extract action features: 2 dimensions [basal, bol]
        self.actions = self.df[["basal", "bolus"]].values.astype(np.float32)
        
        # Extract done flags (1 at episode boundaries, 0 otherwise)
        self.dones = self.df["done"].values.astype(np.float32)
        
        # Define rewards (example: negative absolute deviation from target)
        target_glucose = 0.0  # For normalized glucose, target might be 0 after scaling
        self.rewards = -np.abs(self.states[:, 0] - target_glucose)
        
        # Compute next_states using a vectorized roll
        self.next_states = np.roll(self.states, shift=-1, axis=0)
        
        # For transitions where the current step is an episode end,
        # set the next state to be the current state so that transitions do not cross episodes.
        self.next_states[self.dones == 1] = self.states[self.dones == 1]
        
        # Remove the final row since it doesn't have a valid next state
        self.states = self.states[:-1]
        self.actions = self.actions[:-1]
        self.rewards = self.rewards[:-1]
        self.next_states = self.next_states[:-1]
        self.dones = self.dones[:-1]

    def __len__(self):
        return len(self.states)

    def __getitem__(self, idx):
        # Return a dictionary for the transition (s, a, r, s', done)
        return {
            "state":      torch.tensor(self.states[idx],      dtype=torch.float32),
            "action":     torch.tensor(self.actions[idx],     dtype=torch.float32),
            "reward":     torch.tensor(self.rewards[idx],     dtype=torch.float32),
            "next_state": torch.tensor(self.next_states[idx], dtype=torch.float32),
            "done":       torch.tensor(self.dones[idx],       dtype=torch.float32)
        }


In [6]:

# ===================
# USAGE EXAMPLE
# ===================
if __name__ == "__main__":
    dataset = DiabetesDataset(csv_file="datasets/processed/563-test.csv")
    
    # Peek at the first sample
    sample = dataset[0]
    print("State:", sample["state"])
    print("Action:", sample["action"])
    print("Reward:", sample["reward"])
    print("Next State:", sample["next_state"])
    print("Done:", sample["done"])


State: tensor([ 1.5537,     nan,     nan, -0.0855,     nan,     nan, -1.2715,  0.0000])
Action: tensor([ 3.1836, -0.1611])
Reward: tensor(-1.5537)
Next State: tensor([ 1.5320, -0.1646,     nan, -1.0180, -1.3954,     nan, -1.2561,  0.0000])
Done: tensor(0.)
